In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [19]:
import pickle
import numpy as np
import tensorflow as tf

np.set_printoptions(suppress=True)

In [29]:
dataset = "../../data/snr60.npz"
modelpath = "../gridsearch/noisy_tiny_200k.tf"
scalerpath = "../../data/data_v1.minmax"

with np.load(dataset) as data:
    print("Available variables:", data.files)
    image = data["img"]
    label = data["label"]

X = image[..., np.newaxis]
y_true = label[0]
# Convert flux units to 10^5
y_true[0] /= 1e+5

# Reload the scaler
with open(scalerpath, "rb") as f:
    scaler = pickle.load(f)

# Reload Xception model
model = tf.keras.models.load_model(modelpath)

Available variables: ['img', 'img_nonoise', 'label', 'psf_r', 'psf_img', 'snr', 'sigma']


In [30]:
normalized_preds = model.predict(X)
predictions = scaler.inverse_transform(normalized_preds)
# Convert flux units to 10^5
predictions[:, 0] /= 1e+5

In [31]:
bias = predictions.mean(axis=0) - y_true
bias

array([ 0.02849007,  0.5091243 ,  0.01246777, -0.01230708,  0.00154898],
      dtype=float32)

In [32]:
std = predictions.std(axis=0)
std

array([0.06199119, 0.76120484, 0.03654568, 0.05291573, 0.05689033],
      dtype=float32)